In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.2'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

#show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [11]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_q1 = '''SELECT YEAR(date) AS year_sold, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold'''

spark.sql(avg_price_q1).show()


+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [12]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_q2 = """
    select date_built as year_built, round(avg(price), 2) as avg_price
    from home_sales
    where bedrooms = 3 and bathrooms = 3
    group by year_built
    order by year_built
  """
spark.sql(avg_price_q2).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

avg_price_q3 = """
    select date_built as year_built, round(avg(price), 2) as avg_price
    from home_sales
    where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
    group by year_built
    order by year_built
  """
spark.sql(avg_price_q3).show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_price_q4 = """
    select view, round(avg(price), 2) as avg_price
    from home_sales
    group by view
    having avg(price) >= 35000
    order by view
"""
spark.sql(avg_price_q4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|285069.21|
|   1|288248.11|
|  10|289693.49|
| 100|1026669.5|
|  11|280356.07|
|  12|286575.22|
|  13|285849.95|
|  14|282519.32|
|  15|284907.04|
|  16|291990.83|
|  17|286601.22|
|  18|287532.36|
|  19|280347.62|
|   2|287710.06|
|  20|284820.43|
|  21|283451.36|
|  22|284908.42|
|  23|286977.37|
|  24|281878.86|
|  25|288583.21|
+----+---------+
only showing top 20 rows

--- 0.7127280235290527 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql(avg_price_q4).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|285069.21|
|   1|288248.11|
|  10|289693.49|
| 100|1026669.5|
|  11|280356.07|
|  12|286575.22|
|  13|285849.95|
|  14|282519.32|
|  15|284907.04|
|  16|291990.83|
|  17|286601.22|
|  18|287532.36|
|  19|280347.62|
|   2|287710.06|
|  20|284820.43|
|  21|283451.36|
|  22|284908.42|
|  23|286977.37|
|  24|281878.86|
|  25|288583.21|
+----+---------+
only showing top 20 rows

--- 0.8610763549804688 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('built_partitioned')

In [20]:
# 11. Read the parquet formatted data.
p_df_p = spark.read.parquet('built_partitioned')

In [21]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_built_p')

In [23]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql(avg_price_q4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|285069.21|
|   1|288248.11|
|  10|289693.49|
| 100|1026669.5|
|  11|280356.07|
|  12|286575.22|
|  13|285849.95|
|  14|282519.32|
|  15|284907.04|
|  16|291990.83|
|  17|286601.22|
|  18|287532.36|
|  19|280347.62|
|   2|287710.06|
|  20|284820.43|
|  21|283451.36|
|  22|284908.42|
|  23|286977.37|
|  24|281878.86|
|  25|288583.21|
+----+---------+
only showing top 20 rows

--- 0.90130615234375 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False